In [3]:
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
import re
import hanja
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [4]:
client_id = 'AJb44uAUgBS5u1gIdz0P'
client_secret = 'ThMXBzW9HG'

url = 'https://openapi.naver.com/v1/search/news?'

header = {"X-Naver-Client-Id":client_id,"X-Naver-Client-secret":client_secret}

symbol = '카카오' # 종목
word_list = ['','뉴스','발표','제품','기업','개발','기술','최신','정보','브랜드','이미지','서비스'] # 검색어
display = 100 # 1~100 고정
start = 1 # 1~1000 고정
sort = 'sim' # sim(정확도), date(시간순)

title_list = [] # 기사제목
link_list = [] # 링크
date_list = [] # 날짜
desc_list = [] # 간단설명
for word in word_list:
    keyword = symbol + ' ' + word # 종목 + 검색어
    start = 1
    for _ in range(11):
        params = {'query':keyword , 'display':display, 'start':start, 'sort':sort}
        req = requests.get(url, params = params, headers = header)
        js = req.json()
        title_list += [js['items'][i]['title'] 
                       for i in range(100) 
                       if 'news.naver.com/main' in js['items'][i]['link'] # 네이버뉴스만 추출
                       if symbol in js['items'][i]['title']] # 기사제목에 종목이름 포함만 추출
        link_list += [js['items'][i]['link'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        date_list += [js['items'][i]['pubDate'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        desc_list += [js['items'][i]['description'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        start += 100
        if start == 1001: # 1100개 풀크롤링
            start = 1000
    print(f'{keyword} 검색 뉴스기사 크롤링 완료')

카카오  검색 뉴스기사 크롤링 완료
카카오 뉴스 검색 뉴스기사 크롤링 완료
카카오 발표 검색 뉴스기사 크롤링 완료
카카오 제품 검색 뉴스기사 크롤링 완료
카카오 기업 검색 뉴스기사 크롤링 완료
카카오 개발 검색 뉴스기사 크롤링 완료
카카오 기술 검색 뉴스기사 크롤링 완료
카카오 최신 검색 뉴스기사 크롤링 완료
카카오 정보 검색 뉴스기사 크롤링 완료
카카오 브랜드 검색 뉴스기사 크롤링 완료
카카오 이미지 검색 뉴스기사 크롤링 완료
카카오 서비스 검색 뉴스기사 크롤링 완료


In [5]:
# date_list 포멧 변경
mon_dict = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}    
ymd_list = []
for date in date_list:
    date_split = date.split()
    y = date_split[3]
    m = mon_dict[date_split[2]]
    d = date_split[1]
    ymd = f'{y}-{m}-{d}'
    ymd_list.append(ymd)

week_dict = {'Mon,':'월요일','Tue,':'화요일','Wed,':'수요일','Thu,':'목요일','Fri,':'금요일','Sat,':'토요일','Sun,':'일요일'}
week_list = []
for date in date_list:
    week = week_dict[date.split()[0]]
    week_list.append(week)

In [29]:
# 기사제목과 간단요약 글 전처리
for i in range(len(title_list)):
    sentence = title_list[i]
    sentence = sentence.replace('<b>','').replace('</b>','').replace('&quot;','').replace('&lt;','').replace('&gt;','') # 삭제
    sentence = sentence.replace('↑','상승').replace('↓','하락').replace('%','퍼센트').replace('M&amp;A','인수합병') # 치환
    sentence = hanja.translate(sentence, 'substitution') # 한자 한글로 번역
    title_list[i] = sentence
    
for i in range(len(desc_list)):
    sentence = desc_list[i]
    sentence = sentence.replace('<b>','').replace('</b>','').replace('&quot;','').replace('&lt;','').replace('&gt;','') # 삭제
    sentence = sentence.replace('↑','상승').replace('↓','하락').replace('%','퍼센트').replace('M&amp;A','인수합병') # 치환
    sentence = hanja.translate(sentence, 'substitution') # 한자 한글로 번역
    desc_list[i] = sentence

In [27]:
title_list

['카카오웹툰, 인도네시아 시장 진출',
 "카카오엔터, '카카오웹툰 인도네시아' 출시",
 "카카오모빌리티, 반려동물 이동 서비스 '카카오T 펫' 선봬",
 '카카오웹툰, 인도네시아 진출…아세안으로 확장',
 '카카오엔터, ‘카카오웹툰 인도네시아’ 27일 론칭',
 '카카오웹툰, 인도네시아 서비스 시작합니다',
 '대전시-카카오엔터프라이즈, 디지털 혁신 생태계조성 협약',
 '대전시, 카카오엔터와 손잡고 디지털 혁신 생태계 조성',
 '네이버지도·카카오맵·T맵에서 ‘외래진료센터’ 검색 가능',
 "카카오가 재계 15위…코로나 속 몸집 불린 '빅테크'들",
 "반려동물 이동 서비스 '카카오T 펫' 시험 운영…기본료 8천원",
 '대전시·카카오엔터프라이즈, 디지털 혁신 생태계 조성 협약',
 '카카오엔터프라이즈-카테노이드, 라이브커머스 고도화 협력',
 '카카오웹툰, 대만, 태국 이어 인도네시아 진출…아세안 시장 공략',
 '수백만원 결혼 예물도 카톡 선물…카카오 선물하기 ‘불가리’ 입점',
 "반려동물과 함께 타세요...카카오모빌리티 '카카오 T 펫' 선보여",
 "공정위, '콜 몰아주기' 의혹 카카오모빌리티 제재 절차 착수",
 '카카오, 카테노이드과 제휴…러이브 커머스 시장 공략',
 "인도네시아 진출하는 카카오…'카카오웹툰 인도네시아' 출시",
 '공정위, ‘콜 몰아주기’ 혐의 카카오모빌리티 제재 착수',
 "카카오모빌리티, 반려동물 이동 위한 '카카오 T 펫' 시범 서비스 시작",
 "대전시, 카카오와 디지털 혁신생태계 조성 '맞손'",
 '카카오웹툰 인도네시아 상륙 동남아 먹는다',
 '카카오웹툰 인도네시아 서비스 본격화',
 '확진자 대면진료 병원, 네이버·카카오 등 포털 검색 가능',
 '청와대 관람, 오늘부터 네이버·카카오톡·토스로 신청',
 "카카오톡 선물하기에 '불가리' 입점",
 '잘 데려다주시 개!..댕댕이도 카카오T 탄다',
 '카카오모빌리티, 반려동물 이동 서비스 ‘카카오T 펫’ 출시',
 "카카오모빌리티, 반려동물 이동 위한 '카카오

In [30]:
desc_list

['글로벌 웹툰 플랫폼 카카오웹툰이 27일 인도네시아에서 서비스를 시작한다고 카카오엔터테인먼트가 밝혔다. 카카오엔터는 2018년 12월 인도네시아 웹툰업체 네오바자르를 인수한 뒤 현지 웹툰 서비스인 웹코믹스를... ',
 "카카오엔터테인먼트가 글로벌 웹툰 플랫폼 카카오웹툰을 인도네시아에 출시하며 글로벌 공략을 가속화한다. 한국의 프리미엄 지식재산(IP)과 카카오엔터테인먼트의 해외 성공 노하우를 '카카오웹툰 인도네시아'에... ",
 '카카오모빌리티가 반려동물 이동 서비스 ‘카카오T 펫’을 서울과 인천, 경기 전 지역에서 선보인다고 27일 밝혔다. 베타 서비스로 운행을 시작한 뒤 기사 공급망을 넓혀, 운행 가능 지역을 확대한다는 계획이다. 정식... ',
 '카카오엔터테인먼트는 글로벌 웹툰 플랫폼 카카오웹툰을 인도네시아에 출시한다고 27일 밝혔다. 카카오엔터는 2018년 12월 인도네시아 웹툰업체 네오바자르를 인수한 뒤 현지 웹툰 서비스인 웹코믹스를 카카오페이지... ',
 '카카오엔터테인먼트(대표 이진수, 김성수)가 글로벌 웹툰 플랫폼 카카오웹툰을 인도네시아에 론칭, 글로벌 공략에 불을 붙인다. 한국의 프리미엄 지식재산권(IP)과 카카오엔터테인먼트의 글로벌 성공 노하우를... ',
 '카카오엔터테인먼트(대표 이진수, 김성수)가 글로벌 웹툰 플랫폼 카카오웹툰을 인도네시아에 론칭, 글로벌 공략에 불을 붙인다. 한국의 프리미엄 IP와 카카오엔터테인먼트의 글로벌 성공 노하우를 ‘카카오웹툰... ',
 '기사내용 요약클라우드 센터 조성, 인공지능 인재 양성 맞손 대전시는 27일 시청에서 카카오엔터프라이즈와 인공지능 기반 디지털 혁신 생태계 조성을 위한 업무협약을 체결했다. 협약에 따라 시와 카카오엔터프라이즈는... ',
 '대전시가 카카오엔터프라이즈와 손잡고 대전에 디지털 혁신 생태계를 조성하기로 했다. 대전시는 27일 대전시청에서 카카오엔터프라이즈(백상엽 대표)와 미래기술 통합 플랫폼을 활용한 디지털 혁신 생태계 조성을 위해... ',
 '네이버, 카카오, T맵 등을

In [31]:
df = pd.DataFrame() # DF 생성
df['날짜'] = ymd_list
df['요일'] = week_list
df['기사제목'] = title_list
df['간단설명'] = desc_list
df['링크'] = link_list

df = df.drop_duplicates(['링크']) # 링크 중복 제거
df = df.drop_duplicates(['기사제목']) # 제목 중복 제거

df['날짜'] = pd.to_datetime(df['날짜']) # 날짜열 타입 datetime으로 변경
df = df.sort_values(by=['날짜'], axis=0, ascending = False) # 최신순 정렬
df = df.reset_index(drop=True) # 인덱스 리셋

In [32]:
df

,날짜,요일,기사제목,간단설명,링크
0,2022-04-27,수요일,"카카오웹툰, 인도네시아 시장 진출",글로벌 웹툰 플랫폼 카카오웹툰이 27일 인도네시아에서 서비스를 시작한다고 카카오엔터...,https://news.naver.com/main/read.naver?mode=LS...
1,2022-04-27,수요일,"카카오웹툰, 인도네시아 서비스 시작",카카오엔터테인먼트가 웹툰 플랫폼 카카오웹툰의 인도네시아 서비스를 27일부터 시작한다...,https://news.naver.com/main/read.naver?mode=LS...
2,2022-04-27,수요일,"카카오톡 선물하기, 명품 주얼리 브랜드 ‘불가리’ 입점",카카오는 카카오톡 선물하기 글로벌 명품 주얼리 브랜드 '불가리(BVLGARI)'가 ...,https://news.naver.com/main/read.naver?mode=LS...
3,2022-04-27,수요일,"카카오엔터프라이즈-카테노이드, 미디어 사업 협력",카카오엔터프라이즈(대표 백상엽)는 국내 최대 서비스형 비디오 테크놀로지(VTaaS)...,https://news.naver.com/main/read.naver?mode=LS...
4,2022-04-27,수요일,'대면진료' 가능 병원은?…네이버·카카오 등 검색 서비스,네이버와 카카오로 확진자 대면 진료가 가능한 외래진료센터의 정보를 찾아볼 수 있게 ...,https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...,...,...
2134,2020-06-01,월요일,"카카오키즈 최신 앱, 삼성전자 모바일기기 키즈홈 메인에 기본 탑재",카카오키즈는 삼성전자의 최신 스마트폰과 태블릿PC ‘키즈홈’ 메인 화면에 AI 추천...,https://news.naver.com/main/read.naver?mode=LS...
2135,2020-05-18,월요일,인천시 카카오톡으로 최신 치매 정보 받아보세요,가장 대중적인 SNS매체인 카카오톡 채널을 활용해 최신 치매정보 뿐만 아니라 인천의...,https://news.naver.com/main/read.naver?mode=LS...
2136,2019-10-29,화요일,"카카오키즈, 삼성 '키즈홈'과 제휴… 최신 단말기 기본 탑재",이번 제휴에 따라 삼성 최신 단말기에 ‘카카오키즈앱’이 기본 탑재될 예정이다. 어린...,https://news.naver.com/main/read.naver?mode=LS...
2137,2019-10-28,월요일,"카카오키즈, 삼성전자 '키즈홈' 제휴…최신 단말기에 기본 탑재",카카오키즈가 삼성전자와 어린이 사용자를 위한 '키즈홈' 서비스 제휴를 맺고 최신 단...,https://news.naver.com/main/read.naver?mode=LS...


In [ ]:
# import re
# processing_article = []
# patterns = [r'\([^)]*\)',r'\[[^)]*\]',r'\{[^)]*\}']

# for idx in raw_article:
#     text = idx
#     for pattern in patterns:
#         text = re.sub(pattern=pattern, repl='', string=text)
#     text = re.sub('[↑=+,#/\?:^*\"※&!』…》ⓒ▶️■]','', text)
#     processing_article.append(text)

# processing_article
